In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pprint import pprint

df = pd.read_csv("../src/taxipred/data/taxi_trip_pricing_train.csv")
df.head()




,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
2,30.33,Evening,Weekday,4.0,Low,Unknown,3.48,0.51,0.15,116.81,36.4698
3,8.64,Afternoon,Weekend,2.0,Medium,Clear,2.55,1.71,0.48,89.33,60.2028
4,41.79,Night,Weekend,3.0,High,Clear,4.60,1.77,0.11,86.95,88.1328


In [20]:
X, y = df.drop("Trip_Price", axis="columns"), df["Trip_Price"] 
X = pd.get_dummies(X, drop_first=True)

# Kolla att allt nu är numeriskt
print(X.head())
print(X.dtypes)

X.head(5)

   Trip_Distance_km  Passenger_Count  Base_Fare  Per_Km_Rate  Per_Minute_Rate  \
0             19.35              3.0       3.56         0.80             0.32   
1             36.87              1.0       2.70         1.21             0.15   
2             30.33              4.0       3.48         0.51             0.15   
3              8.64              2.0       2.55         1.71             0.48   
4             41.79              3.0       4.60         1.77             0.11   

   Trip_Duration_Minutes  Time_of_Day_Evening  Time_of_Day_Morning  \
0                  53.82                False                 True   
1                  37.27                 True                False   
2                 116.81                 True                False   
3                  89.33                False                False   
4                  86.95                False                False   

   Time_of_Day_Night  Time_of_Day_Unknown  Day_of_Week_Weekday  \
0              False      

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Time_of_Day_Unknown,Day_of_Week_Weekday,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Traffic_Conditions_Unknown,Weather_Rain,Weather_Snow,Weather_Unknown
0,19.35,3.0,3.56,0.80,0.32,53.82,False,True,False,False,True,False,True,False,False,False,False,False
1,36.87,1.0,2.70,1.21,0.15,37.27,True,False,False,False,False,True,False,False,False,False,False,False
2,30.33,4.0,3.48,0.51,0.15,116.81,True,False,False,False,True,False,True,False,False,False,False,True
3,8.64,2.0,2.55,1.71,0.48,89.33,False,False,False,False,False,True,False,True,False,False,False,False
4,41.79,3.0,4.60,1.77,0.11,86.95,False,False,True,False,False,True,False,False,False,False,False,False


In [15]:
y.head(5)

0    36.2624
1    52.9032
2    36.4698
3    60.2028
4    88.1328
Name: Trip_Price, dtype: float64

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)

In [22]:
scaler = MinMaxScaler()
scaler.fit(X_train)

scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

print(f"{scaled_X_train.min():.2f} ≤ scaled_X_train ≤ {scaled_X_train.max():.2f}")
print(f"{scaled_X_test.min():.2f} ≤ scaled_X_test ≤ {scaled_X_test.max():.2f}")

0.00 ≤ scaled_X_train ≤ 1.00
-0.00 ≤ scaled_X_test ≤ 1.00


In [23]:
model = LinearRegression()
model.fit(scaled_X_train, y_train)

print(f"Parameters: {model.coef_}")
print(f"Intercept parameter: {model.intercept_}")

Parameters: [ 2.65947994e+02 -4.19514344e-01  9.23762576e-01  3.63270907e+01
  2.27498410e+01  3.48301288e+01 -1.77628746e+00 -1.74485516e+00
 -2.06487590e+00 -3.79376139e+00 -7.28836808e-01 -2.29498290e-01
 -2.37963379e+00 -2.07167351e+00 -8.40361409e-01 -3.42595363e+00
 -4.63775470e-01 -2.13874166e+00]
Intercept parameter: -31.92386717279247


In [24]:
test_sample_features = scaled_X_test[0].reshape(1,-1)
test_sample_target = y_test.values[0]

print(f"Scaled features {test_sample_features}, label {test_sample_target}")
print(f"Prediction: {model.predict(test_sample_features)[0]:.2f}")

Scaled features [[0.04536759 0.33333333 0.39597315 0.32       0.425      0.77311584
  0.         0.         1.         0.         0.         1.
  1.         0.         0.         1.         0.         0.        ]], label 36.1464
Prediction: 20.49
